In [1]:
from google.colab import files
uploaded = files.upload()


Saving Hc3Finalfeatures.csv to Hc3Finalfeatures.csv


In [2]:
import pandas as pd

In [3]:
file_path = '/content/Hc3Finalfeatures.csv'
df = pd.read_csv(file_path)


<ipython-input-3-5a10a8c3025b>:2: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [4]:
df['label'] = df['label'].astype(str)

# Clean the data by selecting only 'text' and 'label' columns and dropping NaNs
df_cleaned = df[['text', 'label']].dropna()

# Filter out classes with fewer than a threshold number of samples (e.g., 2)
threshold = 2
class_counts = df_cleaned['label'].value_counts()
to_remove = class_counts[class_counts < threshold].index
df_filtered = df_cleaned[~df_cleaned['label'].isin(to_remove)]

In [5]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [6]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [7]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

In [8]:
df_filtered['processed_text'] = df_filtered['text'].apply(preprocess_text)

<ipython-input-8-390599eb4361>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['processed_text'] = df_filtered['text'].apply(preprocess_text)


In [9]:
# Splitting the data into training and testing sets
from sklearn.model_selection import train_test_split
X = df_filtered['processed_text']
y = df_filtered['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [10]:
# Tokenizing and padding sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 10000
max_len = 200

In [11]:
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)

In [12]:
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

In [13]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len)

In [14]:
# Converting labels to numeric values
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

In [15]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [16]:
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

In [17]:
# Building the RNN model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D

In [18]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [20]:
# Training the model
batch_size = 64
epochs = 10

history = model.fit(X_train_pad, y_train_categorical, epochs=epochs, batch_size=batch_size, validation_data=(X_test_pad, y_test_categorical))

Epoch 1/10
609/609 [==============================] - 555s 903ms/step - loss: 0.1700 - accuracy: 0.9319 - val_loss: 0.1079 - val_accuracy: 0.9624
Epoch 2/10
609/609 [==============================] - 541s 889ms/step - loss: 0.0838 - accuracy: 0.9735 - val_loss: 0.1321 - val_accuracy: 0.9527
Epoch 3/10
609/609 [==============================] - 535s 878ms/step - loss: 0.0464 - accuracy: 0.9852 - val_loss: 0.0891 - val_accuracy: 0.9702
Epoch 4/10
609/609 [==============================] - 539s 886ms/step - loss: 0.0335 - accuracy: 0.9896 - val_loss: 0.1011 - val_accuracy: 0.9714
Epoch 5/10
609/609 [==============================] - 543s 893ms/step - loss: 0.0186 - accuracy: 0.9946 - val_loss: 0.1419 - val_accuracy: 0.9597
Epoch 6/10
609/609 [==============================] - 538s 882ms/step - loss: 0.0161 - accuracy: 0.9953 - val_loss: 0.1268 - val_accuracy: 0.9664
Epoch 7/10
609/609 [==============================] - 547s 898ms/step - loss: 0.0099 - accuracy: 0.9971 - val_loss: 0.1292 -

In [21]:
loss, accuracy = model.evaluate(X_test_pad, y_test_categorical)
print(f'Test Accuracy: {accuracy * 100:.2f}%')

305/305 [==============================] - 29s 93ms/step - loss: 0.1809 - accuracy: 0.9596
Test Accuracy: 95.96%


In [22]:
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

In [23]:
y_pred_probs = model.predict(X_test_pad)
y_pred = np.argmax(y_pred_probs, axis=1)
y_test_labels = np.argmax(y_test_categorical, axis=1)

print(classification_report(y_test_labels, y_pred, target_names=label_encoder.classes_))
print(confusion_matrix(y_test_labels, y_pred))

305/305 [==============================] - 33s 106ms/step
              precision    recall  f1-score   support

           0       0.98      0.94      0.96      4864
           1       0.94      0.98      0.96      4865

    accuracy                           0.96      9729
   macro avg       0.96      0.96      0.96      9729
weighted avg       0.96      0.96      0.96      9729

[[4572  292]
 [ 101 4764]]
